In [ ]:
#hide
! [ -e /content ] && pip install -Uqq fastbook
import fastbook
fastbook.setup_book()

In [ ]:
#hide
from fastbook import *

# Image Classification

## From Dogs and Cats to Pet Breeds

In [ ]:
from fastai.vision.all import *
path = untar_data(URLs.PETS)

In [ ]:
#hide
Path.BASE_PATH = path

In [ ]:
path.ls()

In [ ]:
(path/"images").ls()

In [ ]:
fname = (path/"images").ls()[0]

In [ ]:
re.findall(r'(.+)_\d+.jpg$', fname.name)

In [ ]:
pets = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'),
                 item_tfms=Resize(460),
                 batch_tfms=aug_transforms(size=224, min_scale=0.75))
dls = pets.dataloaders(path/"images")

## Presizing

In [ ]:
dblock1 = DataBlock(blocks=(ImageBlock(), CategoryBlock()),
                   get_y=parent_label,
                   item_tfms=Resize(460))
# Place an image in the 'images/grizzly.jpg' subfolder where this notebook is located before running this
dls1 = dblock1.dataloaders([(Path.cwd()/'images'/'grizzly.jpg')]*100, bs=8)
dls1.train.get_idxs = lambda: Inf.ones
x,y = dls1.valid.one_batch()
_,axs = subplots(1, 2)

x1 = TensorImage(x.clone())
x1 = x1.affine_coord(sz=224)
x1 = x1.rotate(draw=30, p=1.)
x1 = x1.zoom(draw=1.2, p=1.)
x1 = x1.warp(draw_x=-0.2, draw_y=0.2, p=1.)

tfms = setup_aug_tfms([Rotate(draw=30, p=1, size=224), Zoom(draw=1.2, p=1., size=224),
                       Warp(draw_x=-0.2, draw_y=0.2, p=1., size=224)])
x = Pipeline(tfms)(x)
#x.affine_coord(coord_tfm=coord_tfm, sz=size, mode=mode, pad_mode=pad_mode)
TensorImage(x[0]).show(ctx=axs[0])
TensorImage(x1[0]).show(ctx=axs[1]);

### Checking and Debugging a DataBlock

In [ ]:
dls.show_batch(nrows=1, ncols=3)
# shows 3 images of dogs with type. check data with show_batch to see
# if dataset is as expected. 

In [ ]:
pets1 = DataBlock(blocks = (ImageBlock, CategoryBlock),
                 get_items=get_image_files, 
                 splitter=RandomSplitter(seed=42),
                 get_y=using_attr(RegexLabeller(r'(.+)_\d+.jpg$'), 'name'))
pets1.summary(path/"images")

You can see exactly how we gathered the data and split it, how we went from a filename to a *sample* (the tuple (image, category)), then what item transforms were applied and how it failed to collate those samples in a batch (because of the different shapes). 

Now, train a simple model, like in chapter intro. this gets you a baseline result to improve the model later.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2)

# loss is not defined above, but fastai selects an appropriate loss 
# function based on data and model used. 
# in this case, we have image data and a categorical outcome, so fastai 
# will use *cross-entropy loss*

## Cross-Entropy Loss

it's similar to loss function used before. with Two Benefits: 
- It works even when our dependent variable has more than two categories.
- It results in faster and more reliable training.

to see how cross-entropy loss works for dependent variables with more than two categories.

### Viewing Activations and Labels

In [ ]:
x,y = dls.one_batch()

In [ ]:
y # is the dependent variable from a mini-batch. 

In [ ]:
preds,_ = learn.get_preds(dl=[(x,y)]) #get predictions for all images in dataloader
preds[0] #see only predictions for one image. 

In [ ]:
len(preds[0]),preds[0].sum() 
#print length of all predictions, and sum of all predictions
# 37 probabilities all add up to 1.
# To transform the activations of our model into predictions like this, 
# we used something called the *softmax* activation function.
# i.e. transform values from y(activations) to predictions. 

### Softmax

softmasx is an activation fn in final layer to ensure all activations are between 0 and 1, and they add upto 1. similar to sigmoid function.

In [ ]:
plot_function(torch.sigmoid, min=-4,max=4) 
#sigmoid reminder: get values from -x,x to 0->1

In [ ]:
#hide
torch.random.manual_seed(42); #just setting a seed.

In [ ]:
acts = torch.randn((6,2))*2 #multiply by 2 for two categories.
#mock data: we have 6 images and 2 categories(cats and dogs)
acts

In [ ]:
acts.sigmoid()
#taking sigmoid of mock data. 
#this way is not plausible, cause values in a row don't add upto 1.

In [ ]:
(acts[:,0]-acts[:,1]).sigmoid()
# subtracting activation of 7 from 3 and taking sigmoid of that 
# to see if the number is 3. first column  is 3, second(1) is 7.
# a workaround of using sigmoid for two categories.
# but we can't do that for more than two categories, so we have to use softmax.

softmax works for finding probability for more than two categories.
``` python
def softmax(x): return exp(x) / exp(x).sum(dim=1, keepdim=True)
```

In [ ]:
sm_acts = torch.softmax(acts, dim=1)
sm_acts

summary of softmax
`softmax` is the multi-category equivalent of `sigmoid`—we have to use it any time we have more than two categories and the probabilities of the categories must add to 1, and we often use it even when there are just two categories, just to make things a bit more consistent.

**How Softmax Works?**
What does this function do in practice? Taking the exponential ensures all our numbers are positive, and then dividing by the sum ensures we are going to have a bunch of numbers that add up to 1. The exponential also has a nice property: if one of the numbers in our activations `x` is slightly bigger than the others, the exponential will amplify this (since it grows, well... exponentially), which means that in the softmax, that number will be closer to 1. 

Intuitively, the softmax function *really* wants to pick one class among the others, so it's ideal for training a classifier when we know each picture has a definite label. (Note that it may be less ideal during inference, as you might want your model to sometimes tell you it doesn't recognize any of the classes that it has seen during training, and not pick a class because it has a slightly bigger activation score. In this case, it might be better to train a model using multiple binary output columns, each using a sigmoid activation.)

Softmax is the first part of the cross-entropy loss—the second part is log likelihood. 

### Log Likelihood

When we calculated the loss for our MNIST example in the last chapter we used:

```python
def mnist_loss(inputs, targets):
    inputs = inputs.sigmoid()
    return torch.where(targets==1, 1-inputs, inputs).mean()
```
here inputs and 1-inputs were used as losses for binary classification.

when using softmax, its only the matter of selecting the right column for the respective label.

In [ ]:
# for our synthetic 3s and 7s example, let's say these are our labels:
targ = tensor([0,1,0,1,1,0])

In [ ]:
sm_acts #these are softmax activations. 

In [ ]:
idx = range(6)
sm_acts[idx, targ] #matching predictions from activations to targets.

In [ ]:
from IPython.display import HTML
df = pd.DataFrame(sm_acts, columns=["3","7"])
df['targ'] = targ
df['idx'] = idx
df['result'] = sm_acts[range(6), targ]
t = df.style.hide_index()
#To have html code compatible with our script
html = t._repr_html_().split('</style>')[1]
html = re.sub(r'<table id="([^"]+)"\s*>', r'<table >', html)
display(HTML(html))

In [ ]:
-sm_acts[idx, targ]

PyTorch provides a function that does exactly the same thing as `sm_acts[range(n), targ]` (except it takes the negative, because when applying the log afterward, we will have negative numbers), called `nll_loss` (*NLL* stands for *negative log likelihood*):
`nll_loss`  doesn't actually take log, instead it is designed to be used after `log_softmax`. log_softmax combines log and softmax in a fast and accurate way.

In [ ]:
F.nll_loss(sm_acts, targ, reduction='none')

#### Taking the Log

Recall that cross entropy loss may involve the multiplication of many numbers.  Multiplying lots of negative numbers together can cause problems like [numerical underflow](https://en.wikipedia.org/wiki/Arithmetic_underflow) in computers.  Therefore, we want to transform these probabilities to larger values so we can perform mathematical operations on them.  There is a mathematical function that does exactly this: the *logarithm* (available as `torch.log`). It is not defined for numbers less than 0, and looks like this between 0 and 1:

In [ ]:
plot_function(torch.log, min=0,max=1, ty='log(x)', tx='x')

In [ ]:
plot_function(lambda x: -1*torch.log(x), min=0,max=1, tx='x', ty='- log(x)', title = 'Log Loss when true label = 1')

In [ ]:
from IPython.display import HTML
df['loss'] = -torch.log(tensor(df['result']))
t = df.style.hide_index()
#To have html code compatible with our script
html = t._repr_html_().split('</style>')[1]
html = re.sub(r'<table id="([^"]+)"\s*>', r'<table >', html)
display(HTML(html))

### Negative Log Likelihood

When we first take the softmax, and then the log likelihood of that , that combination is called *cross-entropy loss*. in PyTorch, it's available as `nn.CorssEntropyLoss`, which in practice does `log_softmax` and then `nll_loss`.

In [ ]:
loss_func = nn.CrossEntropyLoss() 
#object function instantiated from cross entropy class

In [ ]:
loss_func(acts, targ)

In [ ]:
F.cross_entropy(acts, targ)
#also used like this instead of class instantiating

In [ ]:
nn.CrossEntropyLoss(reduction='none')(acts, targ)
# By default PyTorch loss functions take the mean of the loss of all items.
# You can use `reduction='none'` to disable that:

## Model Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)
interp.plot_confusion_matrix(figsize=(12,12), dpi=60)
#see where model is making a mistake, using a confusion matrix.

In [ ]:
interp.most_confused(min_val=5)
#can't deduce, since too big confusion matrix, so using most_confused function.

## Improving Our Model

### The Learning Rate Finder

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(1, base_lr=0.1)
# 1. start with very small learning rate, use on on mini-batch.
# 2. find losses of the batch, increase LR by some percent(eg. double)
# 3. again calculate loss, and keep doubling until loss starts to increase.
# instead of decreasing.

Pick LR a bit lower than the point where loss starts to increase.
- One order of magnitude less than where the minimum loss was achieved (i.e., the minimum divided by 10)
- The last point where the loss was clearly decreasing 

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
lr_min,lr_steep = learn.lr_find(suggest_funcs=(minimum, steep))

In [ ]:
print(f"Minimum/10: {lr_min:.2e}, steepest point: {lr_steep:.2e}")
# seeing the graph, loss reaches minimun at around 1e-1, but LR of 1e-1 is
# already too high. so picking 3e-3.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fine_tune(2, base_lr=3e-3)

### Unfreezing and Transfer Learning

We discussed briefly in <<chapter_intro>> how transfer learning works. We saw that the basic idea is that a pretrained model, trained potentially on millions of data points (such as ImageNet), is fine-tuned for some other task. But what does this really mean?

We now know that a convolutional neural network consists of many linear layers with a nonlinear activation function between each pair, followed by one or more final linear layers with an activation function such as softmax at the very end. The final linear layer uses a matrix with enough columns such that the output size is the same as the number of classes in our model (assuming that we are doing classification).

This final linear layer is unlikely to be of any use for us when we are fine-tuning in a transfer learning setting, because it is specifically designed to classify the categories in the original pretraining dataset. So when we do transfer learning we remove it, throw it away, and replace it with a new linear layer with the correct number of outputs for our desired task (in this case, there would be 37 activations).

This newly added linear layer will have entirely random weights. Therefore, our model prior to fine-tuning has entirely random outputs. But that does not mean that it is an entirely random model! All of the layers prior to the last one have been carefully trained to be good at image classification tasks in general. As we saw in the images from the [Zeiler and Fergus paper](https://arxiv.org/pdf/1311.2901.pdf) in <<chapter_intro>> (see <<img_layer1>> through <<img_layer4>>), the first few layers encode very general concepts, such as finding gradients and edges, and later layers encode concepts that are still very useful for us, such as finding eyeballs and fur.

We want to train a model in such a way that we allow it to remember all of these generally useful ideas from the pretrained model, use them to solve our particular task (classify pet breeds), and only adjust them as required for the specifics of our particular task.

Our challenge when fine-tuning is to replace the random weights in our added linear layers with weights that correctly achieve our desired task (classifying pet breeds) without breaking the carefully pretrained weights and the other layers. There is actually a very simple trick to allow this to happen: tell the optimizer to only update the weights in those randomly added final layers. Don't change the weights in the rest of the neural network at all. This is called *freezing* those pretrained layers.

In [ ]:
learn.fine_tune?? #see source code for the method.
#trains randomly added layers for one epcoh with all other layers frozen.
# unfreezes all of the layers and trains them for the rest of the epochs defined.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3) #train 3 epochs while freezing other layers.

In [ ]:
learn.unfreeze() #after 3 trainings, we unfreeze the model. 

In [ ]:
learn.lr_find() #find new learning rate, cause we have more layers to traing 
# and weights already trained for 3 epochs.

In [ ]:
learn.fit_one_cycle(6, lr_max=1e-5)

### Discriminative Learning Rates

deepest layers(pretrained imagenet layers) might not need as high a learning rate as the last ones(our added ones), so we must use different learning rates for those, this is called discriminative learning rates.

In [ ]:
learn = vision_learner(dls, resnet34, metrics=error_rate)
learn.fit_one_cycle(3, 3e-3)
learn.unfreeze()
learn.fit_one_cycle(12, lr_max=slice(1e-6,1e-4))
#1e-6 for lowest layers -> low learning rate for pretrained layers
#1e-4 for other layers -> higher learning rate for new layers

In [ ]:
learn.recorder.plot_loss() 
#accuracy matters, valid loss doesn't decrease, but accuracy increases.

### Selecting the Number of Epochs

### Deeper Architectures

In [ ]:
from fastai.callback.fp16 import *
learn = vision_learner(dls, resnet50, metrics=error_rate).to_fp16()
learn.fine_tune(6, freeze_epochs=3)

## Conclusion

## Questionnaire

1. Why do we first resize to a large size on the CPU, and then to a smaller size on the GPU?
1. If you are not familiar with regular expressions, find a regular expression tutorial, and some problem sets, and complete them. Have a look on the book's website for suggestions.
1. What are the two ways in which data is most commonly provided, for most deep learning datasets?
1. Look up the documentation for `L` and try using a few of the new methods that it adds.
1. Look up the documentation for the Python `pathlib` module and try using a few methods of the `Path` class.
1. Give two examples of ways that image transformations can degrade the quality of the data.
1. What method does fastai provide to view the data in a `DataLoaders`?
1. What method does fastai provide to help you debug a `DataBlock`?
1. Should you hold off on training a model until you have thoroughly cleaned your data?
1. What are the two pieces that are combined into cross-entropy loss in PyTorch?
1. What are the two properties of activations that softmax ensures? Why is this important?
1. When might you want your activations to not have these two properties?
1. Calculate the `exp` and `softmax` columns of <<bear_softmax>> yourself (i.e., in a spreadsheet, with a calculator, or in a notebook).
1. Why can't we use `torch.where` to create a loss function for datasets where our label can have more than two categories?
1. What is the value of log(-2)? Why?
1. What are two good rules of thumb for picking a learning rate from the learning rate finder?
1. What two steps does the `fine_tune` method do?
1. In Jupyter Notebook, how do you get the source code for a method or function?
1. What are discriminative learning rates?
1. How is a Python `slice` object interpreted when passed as a learning rate to fastai?
1. Why is early stopping a poor choice when using 1cycle training?
1. What is the difference between `resnet50` and `resnet101`?
1. What does `to_fp16` do?

### Further Research

1. Find the paper by Leslie Smith that introduced the learning rate finder, and read it.
1. See if you can improve the accuracy of the classifier in this chapter. What's the best accuracy you can achieve? Look on the forums and the book's website to see what other students have achieved with this dataset, and how they did it.